# 多臂老虎机 (Multi-Armed Bandits) - 边学边优化

## 引言：拉斯维加斯的智慧

想象你走进拉斯维加斯的赌场，面前有 5 台老虎机。每台机器的赔率不同，但你不知道哪台最好。你有 100 次游戏机会，目标是赚最多的钱。

**策略 A（传统 A/B Testing）**：
- 前 50 次均匀测试每台机器（各 10 次）
- 找出最好的机器
- 后 50 次全押在最好的机器上

**策略 B（Multi-Armed Bandit）**：
- 边玩边学，动态调整
- 好的机器多玩，差的机器少玩
- 但也不完全放弃差的机器（万一运气不好呢？）

这就是 **Explore-Exploit Trade-off** 的核心思想！

---

## 本节目标

1. 理解 MAB 与 A/B Testing 的区别
2. 掌握 Epsilon-Greedy、UCB、Thompson Sampling
3. 学习 Contextual Bandit 处理个性化场景
4. 理解 Regret 遗憾分析
5. 应用于推荐系统、广告投放等场景

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from scipy import stats
from typing import List, Tuple, Dict
import warnings
warnings.filterwarnings('ignore')

# 设置随机种子
np.random.seed(42)

# Plotly 配置
COLORS = {
    'blue': '#2D9CDB',
    'green': '#27AE60',
    'red': '#EB5757',
    'yellow': '#F2C94C',
    'purple': '#9B51E0',
    'gray': '#828282'
}

print("📚 环境准备完成！")

---

# Part 1: MAB vs A/B Testing

## 1.1 A/B Testing 的「遗憾」

### 场景：电商网站按钮颜色测试

假设你在测试 3 种按钮颜色：
- 红色：转化率 5%
- 蓝色：转化率 8%（最优）
- 绿色：转化率 6%

**传统 A/B Testing**：
- 前 3000 个用户均匀分配（各 1000 个）
- 统计分析后选择蓝色
- 后 7000 个用户全部使用蓝色

**问题在哪？**
- 前 3000 个用户中，有 2000 个看到了次优方案
- 损失了 $(8\% - 5.67\%) \times 2000 \approx 46$ 个转化

这就是 **Regret（遗憾）**：

$$
\text{Regret} = \sum_{t=1}^{T} (\mu^* - \mu_{a_t})
$$

其中：
- $T$：总轮数
- $\mu^*$：最优臂的期望收益
- $\mu_{a_t}$：第 $t$ 轮选择的臂的期望收益
- $a_t$：第 $t$ 轮选择的动作

In [ ]:
# 模拟 A/B Testing
class TraditionalABTest:
    def __init__(self, true_rates: List[float], exploration_ratio: float = 0.3):
        """
        Args:
            true_rates: 每个臂的真实转化率
            exploration_ratio: 探索阶段占比
        """
        self.true_rates = np.array(true_rates)
        self.n_arms = len(true_rates)
        self.exploration_ratio = exploration_ratio
        self.best_arm = np.argmax(true_rates)
        
    def run(self, n_rounds: int) -> Tuple[List[int], List[float]]:
        """运行 A/B 测试"""
        actions = []
        rewards = []
        
        exploration_rounds = int(n_rounds * self.exploration_ratio)
        
        # 探索阶段：均匀分配
        for t in range(exploration_rounds):
            arm = t % self.n_arms
            reward = np.random.binomial(1, self.true_rates[arm])
            actions.append(arm)
            rewards.append(reward)
        
        # 利用阶段：全部使用最优臂
        for t in range(exploration_rounds, n_rounds):
            arm = self.best_arm  # 假设我们能找到最优臂
            reward = np.random.binomial(1, self.true_rates[arm])
            actions.append(arm)
            rewards.append(reward)
        
        return actions, rewards

# 运行实验
true_rates = [0.05, 0.08, 0.06]  # 红、蓝、绿
n_rounds = 10000

ab_test = TraditionalABTest(true_rates)
actions, rewards = ab_test.run(n_rounds)

# 计算累积遗憾
best_rate = max(true_rates)
cumulative_regret = []
regret = 0
for action in actions:
    regret += best_rate - true_rates[action]
    cumulative_regret.append(regret)

# 可视化
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=list(range(1, n_rounds + 1)),
    y=cumulative_regret,
    mode='lines',
    name='Cumulative Regret',
    line=dict(color=COLORS['red'], width=2),
    fill='tozeroy',
    fillcolor='rgba(235, 87, 87, 0.1)'
))

# 标注探索阶段
exploration_end = int(n_rounds * 0.3)
fig.add_vline(
    x=exploration_end, 
    line_dash="dash", 
    line_color=COLORS['gray'],
    annotation_text="探索阶段结束",
    annotation_position="top"
)

fig.update_layout(
    title='Traditional A/B Testing: 累积遗憾随时间增长',
    xaxis_title='轮数',
    yaxis_title='累积遗憾（损失的期望收益）',
    template='plotly_white',
    hovermode='x unified',
    height=400
)

fig.show()

print(f"\n📊 A/B Testing 结果：")
print(f"总遗憾：{cumulative_regret[-1]:.2f}")
print(f"平均每轮遗憾：{cumulative_regret[-1] / n_rounds:.4f}")

## 1.2 MAB 的 Explore-Exploit 权衡

### 核心思想

**Exploration（探索）**：尝试不确定的选项，获取信息  
**Exploitation（利用）**：选择当前最优的选项，最大化收益

### 生活中的例子

- **餐厅选择**：去熟悉的餐厅（利用）还是尝试新餐厅（探索）？
- **职业发展**：深耕当前领域（利用）还是学习新技能（探索）？
- **投资理财**：持有蓝筹股（利用）还是投资创业公司（探索）？

### MAB 的优势

1. **动态调整**：根据实时反馈调整策略
2. **减少遗憾**：更快地将流量分配给好的方案
3. **自适应**：环境变化时能快速响应

### 什么时候用 MAB？

✅ **适合 MAB**：
- 流量充足，可以承受动态调整
- 需要快速找到最优方案
- 方案数量较多（>2）
- 收益反馈快速

❌ **不适合 MAB**：
- 需要严格的统计推断（置信区间、p 值）
- 流量有限，需要固定样本量
- 决策后果重大，不能承受风险
- 需要理解「为什么」（MAB 只关注「哪个好"）

---

# Part 2: 基础算法

## 2.1 Epsilon-Greedy：最简单的策略

### 算法思想

- 以概率 $\epsilon$ 随机探索
- 以概率 $1 - \epsilon$ 选择当前最优

### 伪代码

```
for t = 1 to T:
    if random() < epsilon:
        选择随机臂（探索）
    else:
        选择当前平均收益最高的臂（利用）
    
    观察收益，更新统计量
```

### 优缺点

✅ **优点**：简单、直观、易实现  
❌ **缺点**：探索是盲目的，不考虑不确定性

In [ ]:
class EpsilonGreedy:
    def __init__(self, n_arms: int, epsilon: float = 0.1):
        """
        Args:
            n_arms: 臂的数量
            epsilon: 探索概率
        """
        self.n_arms = n_arms
        self.epsilon = epsilon
        self.counts = np.zeros(n_arms)  # 每个臂被选择的次数
        self.values = np.zeros(n_arms)  # 每个臂的平均收益
        
    def select_arm(self) -> int:
        """选择一个臂"""
        if np.random.random() < self.epsilon:
            # 探索：随机选择
            return np.random.randint(self.n_arms)
        else:
            # 利用：选择最优
            return np.argmax(self.values)
    
    def update(self, arm: int, reward: float):
        """更新统计量"""
        self.counts[arm] += 1
        n = self.counts[arm]
        # 增量式更新均值
        self.values[arm] = ((n - 1) / n) * self.values[arm] + (1 / n) * reward
    
    def run(self, true_rates: np.ndarray, n_rounds: int) -> Tuple[List[int], List[float]]:
        """运行实验"""
        actions = []
        rewards = []
        
        for t in range(n_rounds):
            arm = self.select_arm()
            reward = np.random.binomial(1, true_rates[arm])
            self.update(arm, reward)
            actions.append(arm)
            rewards.append(reward)
        
        return actions, rewards

# 运行 Epsilon-Greedy
true_rates = np.array([0.05, 0.08, 0.06])
n_rounds = 10000

eg = EpsilonGreedy(n_arms=3, epsilon=0.1)
eg_actions, eg_rewards = eg.run(true_rates, n_rounds)

# 计算累积遗憾
best_rate = max(true_rates)
eg_regret = []
regret = 0
for action in eg_actions:
    regret += best_rate - true_rates[action]
    eg_regret.append(regret)

print(f"\n🎯 Epsilon-Greedy (ε=0.1) 结果：")
print(f"总遗憾：{eg_regret[-1]:.2f}")
print(f"每个臂的选择次数：{eg.counts}")
print(f"估计的收益：{eg.values}")
print(f"真实收益：{true_rates}")

## 2.2 UCB (Upper Confidence Bound)：乐观探索

### 核心思想

**"Optimism in the Face of Uncertainty"**（面对不确定性保持乐观）

- 不确定性大的臂，给予更高的探索优先级
- 平衡「估计收益」和「不确定性」

### 公式

选择得分最高的臂：

$$
a_t = \arg\max_a \left[ \hat{\mu}_a + \sqrt{\frac{2 \ln t}{n_a}} \right]
$$

其中：
- $\hat{\mu}_a$：臂 $a$ 的平均收益（exploitation）
- $\sqrt{\frac{2 \ln t}{n_a}}$：置信区间半径（exploration bonus）
- $t$：当前轮数
- $n_a$：臂 $a$ 被选择的次数

### 直观理解

- **第一项**（$\hat{\mu}_a$）：已知信息，鼓励利用
- **第二项**（$\sqrt{\frac{2 \ln t}{n_a}}$）：不确定性奖励，鼓励探索
  - $n_a$ 越小，不确定性越大，bonus 越大
  - $t$ 越大，需要更多证据才能确定某个臂是差的

### 生活类比

招聘时：
- **有经验的候选人**：能力已知（$\hat{\mu}$ 高），但天花板也明确（bonus 小）
- **应届毕业生**：潜力未知（$\hat{\mu}$ 低），但可能是天才（bonus 大）

UCB 会给应届生更多机会！

In [ ]:
class UCB:
    def __init__(self, n_arms: int, c: float = 2.0):
        """
        Args:
            n_arms: 臂的数量
            c: 探索系数（默认 sqrt(2) 的平方）
        """
        self.n_arms = n_arms
        self.c = c
        self.counts = np.zeros(n_arms)
        self.values = np.zeros(n_arms)
        self.t = 0  # 总轮数
        
    def select_arm(self) -> int:
        """选择一个臂"""
        # 初始化：每个臂至少选择一次
        for arm in range(self.n_arms):
            if self.counts[arm] == 0:
                return arm
        
        # UCB 得分
        ucb_values = self.values + np.sqrt(
            self.c * np.log(self.t) / self.counts
        )
        return np.argmax(ucb_values)
    
    def update(self, arm: int, reward: float):
        """更新统计量"""
        self.t += 1
        self.counts[arm] += 1
        n = self.counts[arm]
        self.values[arm] = ((n - 1) / n) * self.values[arm] + (1 / n) * reward
    
    def run(self, true_rates: np.ndarray, n_rounds: int) -> Tuple[List[int], List[float]]:
        """运行实验"""
        actions = []
        rewards = []
        
        for t in range(n_rounds):
            arm = self.select_arm()
            reward = np.random.binomial(1, true_rates[arm])
            self.update(arm, reward)
            actions.append(arm)
            rewards.append(reward)
        
        return actions, rewards

# 运行 UCB
ucb = UCB(n_arms=3)
ucb_actions, ucb_rewards = ucb.run(true_rates, n_rounds)

# 计算累积遗憾
ucb_regret = []
regret = 0
for action in ucb_actions:
    regret += best_rate - true_rates[action]
    ucb_regret.append(regret)

print(f"\n🎯 UCB 结果：")
print(f"总遗憾：{ucb_regret[-1]:.2f}")
print(f"每个臂的选择次数：{ucb.counts}")
print(f"估计的收益：{ucb.values}")

## 2.3 Thompson Sampling：贝叶斯思维

### 核心思想

**"Probability Matching"**（概率匹配）

- 为每个臂维护一个收益分布的「信念」
- 从每个臂的信念分布中采样
- 选择采样值最大的臂

### 算法流程（伯努利奖励）

对于每个臂 $a$，维护 Beta 分布 $\text{Beta}(\alpha_a, \beta_a)$：
- $\alpha_a$：成功次数 + 1
- $\beta_a$：失败次数 + 1

每轮：
1. 从每个臂的 Beta 分布中采样：$\theta_a \sim \text{Beta}(\alpha_a, \beta_a)$
2. 选择 $a_t = \arg\max_a \theta_a$
3. 观察奖励 $r_t$，更新：
   - 如果 $r_t = 1$：$\alpha_{a_t} \leftarrow \alpha_{a_t} + 1$
   - 如果 $r_t = 0$：$\beta_{a_t} \leftarrow \beta_{a_t} + 1$

### 为什么用 Beta 分布？

- **Beta 分布**是 [0, 1] 上的连续分布，适合建模概率
- **共轭先验**：Beta 是二项分布的共轭先验，更新公式简单
- **直观解释**：$\text{Beta}(\alpha, \beta)$ 的均值是 $\frac{\alpha}{\alpha + \beta}$，就是成功率

In [ ]:
class ThompsonSampling:
    def __init__(self, n_arms: int):
        """
        Args:
            n_arms: 臂的数量
        """
        self.n_arms = n_arms
        # Beta 分布参数（使用均匀先验 Beta(1, 1)）
        self.alpha = np.ones(n_arms)
        self.beta = np.ones(n_arms)
        
    def select_arm(self) -> int:
        """选择一个臂"""
        # 从每个臂的 Beta 分布中采样
        samples = np.random.beta(self.alpha, self.beta)
        return np.argmax(samples)
    
    def update(self, arm: int, reward: float):
        """更新 Beta 分布参数"""
        if reward == 1:
            self.alpha[arm] += 1
        else:
            self.beta[arm] += 1
    
    def run(self, true_rates: np.ndarray, n_rounds: int) -> Tuple[List[int], List[float]]:
        """运行实验"""
        actions = []
        rewards = []
        
        for t in range(n_rounds):
            arm = self.select_arm()
            reward = np.random.binomial(1, true_rates[arm])
            self.update(arm, reward)
            actions.append(arm)
            rewards.append(reward)
        
        return actions, rewards
    
    def get_mean_estimates(self) -> np.ndarray:
        """获取每个臂的收益估计"""
        return self.alpha / (self.alpha + self.beta)

# 运行 Thompson Sampling
ts = ThompsonSampling(n_arms=3)
ts_actions, ts_rewards = ts.run(true_rates, n_rounds)

# 计算累积遗憾
ts_regret = []
regret = 0
for action in ts_actions:
    regret += best_rate - true_rates[action]
    ts_regret.append(regret)

print(f"\n🎯 Thompson Sampling 结果：")
print(f"总遗憾：{ts_regret[-1]:.2f}")
print(f"Alpha 参数：{ts.alpha}")
print(f"Beta 参数：{ts.beta}")
print(f"估计的收益：{ts.get_mean_estimates()}")
print(f"真实收益：{true_rates}")

### 可视化 Thompson Sampling 的信念演化

In [ ]:
# 重新运行 Thompson Sampling，记录信念演化
ts_evolve = ThompsonSampling(n_arms=3)
snapshots = [100, 500, 1000, 5000, 10000]  # 记录这些时间点的分布
distributions = {t: [] for t in snapshots}

for t in range(1, n_rounds + 1):
    arm = ts_evolve.select_arm()
    reward = np.random.binomial(1, true_rates[arm])
    ts_evolve.update(arm, reward)
    
    if t in snapshots:
        distributions[t] = {
            'alpha': ts_evolve.alpha.copy(),
            'beta': ts_evolve.beta.copy()
        }

# 绘制信念演化
fig = make_subplots(
    rows=2, cols=3,
    subplot_titles=[f't = {t}' for t in snapshots],
    vertical_spacing=0.15,
    horizontal_spacing=0.1
)

x = np.linspace(0, 0.15, 1000)
arm_names = ['红色 (5%)', '蓝色 (8%)', '绿色 (6%)']
colors_list = [COLORS['red'], COLORS['blue'], COLORS['green']]

for idx, t in enumerate(snapshots):
    row = idx // 3 + 1
    col = idx % 3 + 1
    
    alpha = distributions[t]['alpha']
    beta = distributions[t]['beta']
    
    for arm in range(3):
        y = stats.beta.pdf(x, alpha[arm], beta[arm])
        fig.add_trace(
            go.Scatter(
                x=x, y=y,
                mode='lines',
                name=arm_names[arm],
                line=dict(color=colors_list[arm], width=2),
                showlegend=(idx == 0),
                legendgroup=f'arm{arm}'
            ),
            row=row, col=col
        )
        
        # 标注真实值
        fig.add_vline(
            x=true_rates[arm],
            line_dash="dash",
            line_color=colors_list[arm],
            line_width=1,
            row=row, col=col
        )

fig.update_xaxes(title_text="转化率", range=[0, 0.15])
fig.update_yaxes(title_text="概率密度")

fig.update_layout(
    title='Thompson Sampling 信念演化（虚线为真实值）',
    template='plotly_white',
    height=600,
    showlegend=True
)

fig.show()

print("\n💡 观察：")
print("- 随着时间推移，分布越来越窄（不确定性降低）")
print("- 蓝色的分布最终集中在 8% 附近")
print("- 红色和绿色的分布被探索得更少，但也逐渐收敛")

## 2.4 算法对比

In [ ]:
# 对比三种算法和 A/B Testing
fig = go.Figure()

algorithms = [
    ('A/B Testing', cumulative_regret, COLORS['gray']),
    ('Epsilon-Greedy', eg_regret, COLORS['yellow']),
    ('UCB', ucb_regret, COLORS['purple']),
    ('Thompson Sampling', ts_regret, COLORS['blue'])
]

for name, regret, color in algorithms:
    fig.add_trace(go.Scatter(
        x=list(range(1, len(regret) + 1)),
        y=regret,
        mode='lines',
        name=name,
        line=dict(color=color, width=2)
    ))

fig.update_layout(
    title='算法对比：累积遗憾随时间变化',
    xaxis_title='轮数',
    yaxis_title='累积遗憾',
    template='plotly_white',
    hovermode='x unified',
    height=500
)

fig.show()

# 统计表
results_df = pd.DataFrame({
    '算法': ['A/B Testing', 'Epsilon-Greedy', 'UCB', 'Thompson Sampling'],
    '总遗憾': [
        cumulative_regret[-1],
        eg_regret[-1],
        ucb_regret[-1],
        ts_regret[-1]
    ],
    '平均遗憾': [
        cumulative_regret[-1] / n_rounds,
        eg_regret[-1] / n_rounds,
        ucb_regret[-1] / n_rounds,
        ts_regret[-1] / n_rounds
    ]
})

results_df['相对 A/B Testing'] = (
    (results_df['总遗憾'] / results_df.loc[0, '总遗憾'] - 1) * 100
).round(1).astype(str) + '%'

print("\n📊 算法对比结果：")
print(results_df.to_string(index=False))

print("\n🎯 结论：")
print("- Thompson Sampling 表现最好，遗憾最小")
print("- UCB 次之，理论保证强")
print("- Epsilon-Greedy 简单但不够高效")
print("- A/B Testing 遗憾最大，因为探索阶段固定")

### 💡 TODO 1: 调整 Epsilon-Greedy 的探索率

尝试不同的 $\epsilon$ 值（0.05, 0.1, 0.2），观察对遗憾的影响。

In [ ]:
# 答案：测试不同的 epsilon 值
epsilon_values = [0.05, 0.1, 0.2]
results = {}

for eps in epsilon_values:
    eg_test = EpsilonGreedy(n_arms=3, epsilon=eps)
    actions, rewards = eg_test.run(true_rates, n_rounds)
    
    # 计算累积遗憾
    regret = []
    cumulative = 0
    for action in actions:
        cumulative += best_rate - true_rates[action]
        regret.append(cumulative)
    
    results[eps] = {
        'regret': regret,
        'total': regret[-1],
        'counts': eg_test.counts
    }

# 可视化对比
fig = go.Figure()

for eps in epsilon_values:
    fig.add_trace(go.Scatter(
        x=list(range(1, n_rounds + 1)),
        y=results[eps]['regret'],
        mode='lines',
        name=f'ε = {eps}',
        line=dict(width=2)
    ))

fig.update_layout(
    title='不同 Epsilon 值的遗憾对比',
    xaxis_title='轮数',
    yaxis_title='累积遗憾',
    template='plotly_white',
    height=500
)

fig.show()

print("\n📊 不同 Epsilon 值的结果对比：")
for eps in epsilon_values:
    print(f"\nε = {eps}:")
    print(f"  总遗憾: {results[eps]['total']:.2f}")
    print(f"  每个臂选择次数: {results[eps]['counts']}")

print("\n💡 分析：")
print("- ε 太小 (0.05): 探索不足，可能错过最优臂")
print("- ε 太大 (0.2): 过度探索，浪费在次优臂上")
print("- ε = 0.1: 在本例中表现较好，平衡了探索和利用")

---

# Part 3: Contextual Bandit

## 3.1 从 MAB 到 Contextual Bandit

### 经典 MAB 的局限

- **假设**：所有用户相同，最优臂是固定的
- **现实**：不同用户偏好不同
  - 年轻人喜欢蓝色按钮
  - 老年人喜欢红色按钮

### Contextual Bandit

**核心思想**：根据上下文（用户特征）选择最优臂

- **输入**：上下文 $x_t \in \mathbb{R}^d$（用户年龄、性别、地域等）
- **动作**：选择臂 $a_t$
- **反馈**：观察奖励 $r_t$
- **目标**：学习策略 $\pi(x) \to a$

### 应用场景

- **个性化推荐**：根据用户历史推荐商品
- **新闻推送**：根据阅读偏好推送文章
- **广告投放**：根据用户画像展示广告

## 3.2 LinUCB 算法

### 核心假设

奖励与上下文的关系是**线性的**：

$$
r_{t,a} = x_t^\top \theta_a + \epsilon_t
$$

其中：
- $x_t \in \mathbb{R}^d$：上下文向量
- $\theta_a \in \mathbb{R}^d$：臂 $a$ 的参数向量
- $\epsilon_t$：噪声

### 算法流程

对于每个臂 $a$，维护：
- $A_a = I_d + \sum_{t: a_t=a} x_t x_t^\top$ （设计矩阵）
- $b_a = \sum_{t: a_t=a} r_t x_t$ （奖励向量）

在时刻 $t$：
1. 计算每个臂的 UCB：

$$
\text{UCB}_a(x_t) = \hat{\theta}_a^\top x_t + \alpha \sqrt{x_t^\top A_a^{-1} x_t}
$$

   其中 $\hat{\theta}_a = A_a^{-1} b_a$（岭回归估计）

2. 选择 $a_t = \arg\max_a \text{UCB}_a(x_t)$
3. 观察奖励 $r_t$，更新 $A_{a_t}$ 和 $b_{a_t}$

### 直观解释

- **第一项**（$\hat{\theta}_a^\top x_t$）：预测奖励
- **第二项**（$\alpha \sqrt{x_t^\top A_a^{-1} x_t}$）：不确定性（越少见的上下文，不确定性越大）

In [ ]:
class LinUCB:
    def __init__(self, n_arms: int, n_features: int, alpha: float = 1.0):
        """
        Args:
            n_arms: 臂的数量
            n_features: 特征维度
            alpha: 探索系数
        """
        self.n_arms = n_arms
        self.n_features = n_features
        self.alpha = alpha
        
        # 为每个臂初始化参数
        self.A = [np.identity(n_features) for _ in range(n_arms)]
        self.b = [np.zeros(n_features) for _ in range(n_arms)]
    
    def select_arm(self, context: np.ndarray) -> int:
        """根据上下文选择臂"""
        ucb_values = []
        
        for arm in range(self.n_arms):
            # 计算参数估计
            A_inv = np.linalg.inv(self.A[arm])
            theta = A_inv @ self.b[arm]
            
            # 计算 UCB
            ucb = theta @ context + self.alpha * np.sqrt(
                context @ A_inv @ context
            )
            ucb_values.append(ucb)
        
        return np.argmax(ucb_values)
    
    def update(self, arm: int, context: np.ndarray, reward: float):
        """更新参数"""
        self.A[arm] += np.outer(context, context)
        self.b[arm] += reward * context
    
    def get_theta(self, arm: int) -> np.ndarray:
        """获取臂的参数估计"""
        return np.linalg.inv(self.A[arm]) @ self.b[arm]

# 模拟场景：不同年龄用户对按钮颜色的偏好
# 假设特征：[年龄 / 100, 性别(0/1)]
# 真实参数：
# - 红色：年轻人不喜欢，男性更喜欢
# - 蓝色：年轻人喜欢，性别无关
# - 绿色：老年人喜欢，女性更喜欢

true_theta = np.array([
    [-0.2, 0.1],   # 红色
    [0.3, 0.0],    # 蓝色
    [0.2, -0.1]    # 绿色
])

def generate_context() -> np.ndarray:
    """生成随机上下文"""
    age = np.random.uniform(18, 70) / 100
    gender = np.random.randint(2)
    return np.array([age, gender])

def get_reward(arm: int, context: np.ndarray) -> float:
    """根据上下文和臂生成奖励"""
    mean_reward = context @ true_theta[arm]
    # 映射到 [0, 1]
    prob = 1 / (1 + np.exp(-mean_reward * 10))
    return np.random.binomial(1, prob)

# 运行 LinUCB
n_rounds = 5000
linucb = LinUCB(n_arms=3, n_features=2, alpha=0.5)

linucb_actions = []
linucb_rewards = []
contexts = []

for t in range(n_rounds):
    context = generate_context()
    arm = linucb.select_arm(context)
    reward = get_reward(arm, context)
    
    linucb.update(arm, context, reward)
    
    linucb_actions.append(arm)
    linucb_rewards.append(reward)
    contexts.append(context)

# 打印学到的参数
print("\n🎯 LinUCB 学习结果：")
print("\n真实参数 θ:")
print("红色:", true_theta[0])
print("蓝色:", true_theta[1])
print("绿色:", true_theta[2])

print("\n学习到的参数 θ:")
for arm in range(3):
    arm_names = ['红色', '蓝色', '绿色']
    print(f"{arm_names[arm]}:", linucb.get_theta(arm))

print(f"\n平均奖励: {np.mean(linucb_rewards):.3f}")
print(f"每个臂的选择次数: {np.bincount(linucb_actions)}")

### 可视化：不同用户群体的最优臂

In [ ]:
# 生成网格上的预测
age_range = np.linspace(0.18, 0.70, 50)
gender_vals = [0, 1]

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=['男性用户', '女性用户']
)

arm_names = ['红色', '蓝色', '绿色']
colors_list = [COLORS['red'], COLORS['blue'], COLORS['green']]

for gender_idx, gender in enumerate(gender_vals):
    for arm in range(3):
        theta = linucb.get_theta(arm)
        predicted_rewards = []
        
        for age in age_range:
            context = np.array([age, gender])
            reward = context @ theta
            predicted_rewards.append(reward)
        
        fig.add_trace(
            go.Scatter(
                x=age_range * 100,
                y=predicted_rewards,
                mode='lines',
                name=arm_names[arm],
                line=dict(color=colors_list[arm], width=2),
                showlegend=(gender_idx == 0),
                legendgroup=f'arm{arm}'
            ),
            row=1, col=gender_idx + 1
        )

fig.update_xaxes(title_text="年龄")
fig.update_yaxes(title_text="预测奖励")

fig.update_layout(
    title='LinUCB 学习到的策略：不同年龄和性别的最优按钮',
    template='plotly_white',
    height=400
)

fig.show()

print("\n💡 观察：")
print("- 对于男性用户，年轻人更喜欢蓝色，年长者偏好绿色")
print("- 对于女性用户，整体偏好略有不同")
print("- 这就是个性化推荐的基础！")

## 3.3 神经网络方法（Neural Contextual Bandit）

### LinUCB 的局限

- **线性假设**过强：真实世界中，奖励与特征的关系往往是非线性的
- **特征工程**：需要手工设计特征

### Neural Bandit

用神经网络 $f_\theta(x, a)$ 建模奖励：

$$
r_{t,a} = f_\theta(x_t, a) + \epsilon_t
$$

### 挑战：如何探索？

1. **Epsilon-Greedy + NN**：简单但低效
2. **Dropout as Bayesian Approximation**：用 Dropout 估计不确定性
3. **Bootstrap**：训练多个网络，用分歧度量不确定性
4. **Neural Linear (NeuralLinear)**：结合 NN 和 LinUCB

### NeuralLinear 思想

1. 用神经网络提取特征：$\phi(x) = \text{NN}(x)$
2. 在特征空间上用 LinUCB：$r = \phi(x)^\top \theta_a$

这里我们实现一个简化版本。

### 💡 TODO 2: 实现 Epsilon-Greedy + Neural Network

使用简单的神经网络（如 2 层 MLP）预测奖励，结合 Epsilon-Greedy 探索。

In [ ]:
# 答案：Neural Epsilon-Greedy 实现
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler

class NeuralEpsilonGreedy:
    def __init__(self, n_arms: int, n_features: int, epsilon: float = 0.1,
                 retrain_interval: int = 100):
        self.n_arms = n_arms
        self.n_features = n_features
        self.epsilon = epsilon
        self.retrain_interval = retrain_interval
        
        # 为每个臂训练一个模型
        self.models = [MLPRegressor(hidden_layer_sizes=(32, 16), 
                                    max_iter=100, random_state=42)
                       for _ in range(n_arms)]
        self.scalers = [StandardScaler() for _ in range(n_arms)]
        
        # 存储训练数据
        self.data = {i: {'X': [], 'y': []} for i in range(n_arms)}
        self.t = 0
    
    def select_arm(self, context: np.ndarray) -> int:
        # 前期随机探索以收集初始数据
        if self.t < self.n_arms * 10:
            return self.t % self.n_arms
        
        # Epsilon-Greedy
        if np.random.random() < self.epsilon:
            return np.random.randint(self.n_arms)
        else:
            # 预测每个臂的奖励
            predictions = []
            for arm in range(self.n_arms):
                if len(self.data[arm]['X']) > 10:
                    X_scaled = self.scalers[arm].transform([context])
                    pred = self.models[arm].predict(X_scaled)[0]
                else:
                    pred = 0
                predictions.append(pred)
            return np.argmax(predictions)
    
    def update(self, arm: int, context: np.ndarray, reward: float):
        self.t += 1
        self.data[arm]['X'].append(context)
        self.data[arm]['y'].append(reward)
        
        # 定期重新训练
        if self.t % self.retrain_interval == 0:
            for a in range(self.n_arms):
                if len(self.data[a]['X']) > 10:
                    X = np.array(self.data[a]['X'])
                    y = np.array(self.data[a]['y'])
                    self.scalers[a].fit(X)
                    X_scaled = self.scalers[a].transform(X)
                    self.models[a].fit(X_scaled, y)

# 运行 Neural Epsilon-Greedy
n_rounds_neural = 3000
neural_eg = NeuralEpsilonGreedy(n_arms=3, n_features=2, epsilon=0.1, retrain_interval=100)

neural_actions = []
neural_rewards = []

for t in range(n_rounds_neural):
    context = generate_context()
    arm = neural_eg.select_arm(context)
    reward = get_reward(arm, context)
    neural_eg.update(arm, context, reward)
    neural_actions.append(arm)
    neural_rewards.append(reward)

print("\n🎯 Neural Epsilon-Greedy 结果：")
print(f"平均奖励: {np.mean(neural_rewards):.3f}")
print(f"每个臂选择次数: {np.bincount(neural_actions)}")
print(f"\n与 LinUCB 对比:")
print(f"LinUCB 平均奖励: {np.mean(linucb_rewards):.3f}")
print(f"Neural-EG 平均奖励: {np.mean(neural_rewards):.3f}")
print("\n💡 Neural Bandit 可以捕捉非线性关系，但需要更多数据和计算资源")

---

# Part 4: 遗憾分析 (Regret Analysis)

## 4.1 Regret 的定义

### 累积遗憾（Cumulative Regret）

$$
R_T = \sum_{t=1}^{T} (\mu^* - \mu_{a_t}) = T \mu^* - \sum_{t=1}^{T} \mu_{a_t}
$$

其中：
- $\mu^* = \max_a \mu_a$：最优臂的期望收益
- $\mu_{a_t}$：第 $t$ 轮选择的臂的期望收益

### 直观理解

Regret 衡量「如果一直选最优臂」和「实际选择」的差距。

- **最好的情况**：$R_T = 0$（一直选最优臂）
- **最坏的情况**：$R_T = T \Delta$（一直选最差臂，$\Delta$ 是与最优臂的差距）

## 4.2 各算法的 Regret Bound

### Epsilon-Greedy

$$
R_T = O(T^{2/3})
$$

- **线性遗憾**：探索永远不停止，即使已经找到最优臂
- **改进**：Epsilon-Decreasing（$\epsilon_t = 1/t$）可以达到 $O(\log T)$

### UCB

$$
R_T = O\left( \sqrt{KT \log T} \right)
$$

其中 $K$ 是臂的数量。

- **次线性遗憾**：随时间推移，平均遗憾 $R_T / T \to 0$
- **理论保证强**：对任何奖励分布都成立

### Thompson Sampling

$$
R_T = O\left( \sqrt{KT} \right)
$$

- **实践中表现最好**
- **理论分析较难**（依赖于先验分布）

## 4.3 Gap-Dependent Bound

定义 **Gap**：$\Delta_a = \mu^* - \mu_a$（臂 $a$ 与最优臂的差距）

UCB 的 Gap-Dependent Bound：

$$
R_T = O\left( \sum_{a: \Delta_a > 0} \frac{\log T}{\Delta_a} \right)
$$

**直观解释**：
- Gap 越大的臂（$\Delta_a$ 大），越容易被识别为次优，探索次数少
- Gap 越小的臂（$\Delta_a$ 小），需要更多次探索才能确定

## 4.4 Lower Bound（不可逾越的界限）

Lai & Robbins (1985) 证明，**任何算法**的遗憾至少是：

$$
R_T = \Omega\left( \sum_{a: \Delta_a > 0} \frac{\log T}{\Delta_a} \right)
$$

这意味着 UCB 和 Thompson Sampling 在 Gap-Dependent 意义下是**渐近最优的**。

In [ ]:
# 实验：验证 Regret Bound
# 运行多次实验，观察平均遗憾的增长趋势

n_experiments = 50
n_rounds = 10000
true_rates = np.array([0.3, 0.5, 0.4])  # Gap 更明显的设置

ucb_regrets = []
ts_regrets = []

for exp in range(n_experiments):
    # UCB
    ucb = UCB(n_arms=3)
    ucb_actions, _ = ucb.run(true_rates, n_rounds)
    regret = sum(max(true_rates) - true_rates[a] for a in ucb_actions)
    ucb_regrets.append(regret)
    
    # Thompson Sampling
    ts = ThompsonSampling(n_arms=3)
    ts_actions, _ = ts.run(true_rates, n_rounds)
    regret = sum(max(true_rates) - true_rates[a] for a in ts_actions)
    ts_regrets.append(regret)

# 绘制分布
fig = go.Figure()

fig.add_trace(go.Box(
    y=ucb_regrets,
    name='UCB',
    marker_color=COLORS['purple']
))

fig.add_trace(go.Box(
    y=ts_regrets,
    name='Thompson Sampling',
    marker_color=COLORS['blue']
))

fig.update_layout(
    title=f'Regret 分布（{n_experiments} 次实验，每次 {n_rounds} 轮）',
    yaxis_title='累积遗憾',
    template='plotly_white',
    height=400
)

fig.show()

print(f"\n📊 统计结果（{n_experiments} 次实验）:")
print(f"UCB 平均遗憾: {np.mean(ucb_regrets):.2f} ± {np.std(ucb_regrets):.2f}")
print(f"Thompson Sampling 平均遗憾: {np.mean(ts_regrets):.2f} ± {np.std(ts_regrets):.2f}")

# 理论预测
K = 3
T = n_rounds
theoretical_ucb = np.sqrt(K * T * np.log(T))
theoretical_ts = np.sqrt(K * T)

print(f"\n📐 理论 Bound:")
print(f"UCB: O(√(KT log T)) ≈ {theoretical_ucb:.2f}")
print(f"Thompson Sampling: O(√(KT)) ≈ {theoretical_ts:.2f}")
print(f"\n实际遗憾远小于理论上界，说明 Bound 是宽松的。")

## 4.5 实践中的选择

### 算法选择指南

| 场景 | 推荐算法 | 理由 |
|-----|---------|-----|
| **快速原型** | Epsilon-Greedy | 简单易实现 |
| **需要理论保证** | UCB | Bound 清晰，无需先验 |
| **追求最优性能** | Thompson Sampling | 实践中最好 |
| **有上下文信息** | LinUCB | 个性化推荐 |
| **非线性关系** | Neural Bandit | 复杂模式识别 |

### 超参数调优

- **Epsilon-Greedy**：$\epsilon \in [0.05, 0.2]$
- **UCB**：$c \in [1, 3]$（$c=2$ 是标准选择）
- **LinUCB**：$\alpha \in [0.1, 2]$（需要交叉验证）

### A/B Testing vs MAB 决策树

```
需要严格的统计推断（p 值、置信区间）？
├─ 是 → A/B Testing
└─ 否 → 流量是否充足？
    ├─ 否 → A/B Testing
    └─ 是 → 是否有上下文信息？
        ├─ 是 → Contextual Bandit (LinUCB)
        └─ 否 → Multi-Armed Bandit (Thompson Sampling)
```

---

# Part 5: 业务应用

## 5.1 推荐系统中的应用

### 场景：新闻推荐

- **问题**：给用户推荐哪篇文章？
- **挑战**：
  - 文章数量多（数千篇）
  - 用户偏好不同
  - 文章时效性强（新文章不断出现）

### 解决方案：Contextual Bandit

- **上下文**：用户特征（历史阅读、兴趣标签）+ 文章特征（类别、标题）
- **动作**：选择推荐哪篇文章
- **奖励**：点击、阅读时长

### 真实案例

- **Yahoo! News**：LinUCB 提升 CTR 12.5%
- **Google News**：Neural Bandit 大规模应用

## 5.2 广告投放

### 场景：广告位竞价

- **问题**：给这个用户展示哪个广告？
- **目标**：最大化点击率 × 出价

### 挑战

- **延迟反馈**：广告点击后的转化可能几天后才发生
- **预算约束**：某些广告的预算有限
- **非平稳性**：用户兴趣随时间变化

### 解决方案

- **Sliding Window**：只使用最近的数据
- **Discounted UCB**：给近期数据更高权重
- **Budget-Constrained Bandit**：考虑预算的 Bandit

## 5.3 动态定价

### 场景：电商动态调价

- **问题**：某商品应该定价多少？
- **动作**：选择价格区间（如 99, 109, 119）
- **奖励**：销量 × 利润率

### 方法

- **连续臂 Bandit**：价格是连续变量
- **GP-UCB (Gaussian Process UCB)**：用高斯过程建模奖励函数

## 5.4 临床试验

### 场景：药物试验

传统 RCT（随机对照试验）：
- 前期固定分配，后期分析
- **伦理问题**：明知某药效果差，还要给患者使用

### Adaptive Trial (自适应试验)

用 Thompson Sampling：
- 动态调整分配比例
- 更多患者接受有效治疗
- FDA 已批准多个自适应试验

### 真实案例

- **BATTLE Trial** (肺癌治疗)：用自适应设计，提高治疗有效率
- **COVID-19 疫苗试验**：部分采用自适应方法

In [ ]:
# 案例模拟：新闻推荐系统

class NewsRecommendation:
    def __init__(self, n_articles: int = 10):
        """
        Args:
            n_articles: 文章数量
        """
        self.n_articles = n_articles
        
        # 每篇文章的真实质量（用户不知道）
        self.true_ctr = np.random.uniform(0.01, 0.10, n_articles)
        self.true_ctr[0] = 0.12  # 设置一篇爆款文章
        
        # 用 Thompson Sampling 推荐
        self.ts = ThompsonSampling(n_articles)
        
    def simulate(self, n_users: int = 10000):
        """模拟推荐过程"""
        actions = []
        rewards = []
        cumulative_ctr = []
        
        for t in range(n_users):
            # 推荐文章
            article = self.ts.select_arm()
            
            # 用户是否点击
            click = np.random.binomial(1, self.true_ctr[article])
            
            # 更新模型
            self.ts.update(article, click)
            
            actions.append(article)
            rewards.append(click)
            cumulative_ctr.append(np.mean(rewards))
        
        return actions, rewards, cumulative_ctr

# 运行模拟
news_system = NewsRecommendation(n_articles=10)
actions, rewards, cumulative_ctr = news_system.simulate(n_users=10000)

# 可视化
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=['累积点击率', '各文章推荐次数']
)

# 累积 CTR
fig.add_trace(
    go.Scatter(
        x=list(range(1, len(cumulative_ctr) + 1)),
        y=cumulative_ctr,
        mode='lines',
        line=dict(color=COLORS['blue'], width=2),
        name='实际 CTR'
    ),
    row=1, col=1
)

# 最优 CTR（如果一直推荐最好的文章）
best_ctr = max(news_system.true_ctr)
fig.add_hline(
    y=best_ctr,
    line_dash="dash",
    line_color=COLORS['green'],
    annotation_text=f"最优 CTR: {best_ctr:.3f}",
    row=1, col=1
)

# 各文章推荐次数
article_counts = np.bincount(actions, minlength=news_system.n_articles)
fig.add_trace(
    go.Bar(
        x=[f'文章{i}' for i in range(news_system.n_articles)],
        y=article_counts,
        marker_color=COLORS['purple'],
        showlegend=False
    ),
    row=1, col=2
)

fig.update_xaxes(title_text="用户数", row=1, col=1)
fig.update_xaxes(title_text="文章", row=1, col=2)
fig.update_yaxes(title_text="点击率", row=1, col=1)
fig.update_yaxes(title_text="推荐次数", row=1, col=2)

fig.update_layout(
    title='新闻推荐系统模拟',
    template='plotly_white',
    height=400
)

fig.show()

print("\n📊 推荐系统结果：")
print(f"最终 CTR: {cumulative_ctr[-1]:.4f}")
print(f"最优 CTR: {best_ctr:.4f}")
print(f"达到最优的 {cumulative_ctr[-1]/best_ctr*100:.1f}%")

print("\n各文章真实 CTR 和推荐次数:")
for i in range(news_system.n_articles):
    print(f"文章{i}: CTR={news_system.true_ctr[i]:.4f}, 推荐{article_counts[i]}次")

### 💡 TODO 3: 模拟广告投放场景

设计一个广告投放系统：
- 5 个广告，不同的 CTR 和出价
- 目标：最大化 CTR × 出价
- 使用 UCB 或 Thompson Sampling

In [ ]:
# 答案：广告投放模拟
class AdPlacement:
    def __init__(self, n_ads: int = 5):
        # 每个广告的真实 CTR 和出价
        np.random.seed(42)
        self.true_ctrs = np.random.uniform(0.01, 0.08, n_ads)
        self.bids = np.random.uniform(0.5, 2.0, n_ads)  # 出价（元）
        
        # 真实收益 = CTR × 出价
        self.true_revenues = self.true_ctrs * self.bids
        self.best_ad = np.argmax(self.true_revenues)
        
        print("📊 广告设置：")
        for i in range(n_ads):
            print(f"广告 {i}: CTR={self.true_ctrs[i]:.3f}, 出价=¥{self.bids[i]:.2f}, "
                  f"期望收益=¥{self.true_revenues[i]:.4f}")
        print(f"\n最优广告: {self.best_ad} (期望收益: ¥{self.true_revenues[self.best_ad]:.4f})")
    
    def get_reward(self, ad: int) -> float:
        # 用户点击与否
        click = np.random.binomial(1, self.true_ctrs[ad])
        # 收益 = 点击 × 出价
        return click * self.bids[ad]

# 创建广告系统
ad_system = AdPlacement(n_ads=5)

# 策略 1: Thompson Sampling
ts_ads = ThompsonSampling(n_arms=5)
ts_actions = []
ts_revenues = []

for t in range(10000):
    ad = ts_ads.select_arm()
    revenue = ad_system.get_reward(ad)
    # Thompson Sampling 需要 0/1 奖励，这里用点击作为奖励
    click = 1 if revenue > 0 else 0
    ts_ads.update(ad, click)
    ts_actions.append(ad)
    ts_revenues.append(revenue)

# 策略 2: 均匀推荐
uniform_revenues = []
for t in range(10000):
    ad = t % 5
    revenue = ad_system.get_reward(ad)
    uniform_revenues.append(revenue)

# 对比结果
print("\n\n📊 广告投放结果对比：")
print(f"\nThompson Sampling:")
print(f"  总收益: ¥{sum(ts_revenues):.2f}")
print(f"  平均收益: ¥{np.mean(ts_revenues):.4f}")
print(f"  各广告展示次数: {np.bincount(ts_actions)}")

print(f"\n均匀推荐:")
print(f"  总收益: ¥{sum(uniform_revenues):.2f}")
print(f"  平均收益: ¥{np.mean(uniform_revenues):.4f}")

improvement = (sum(ts_revenues) - sum(uniform_revenues)) / sum(uniform_revenues) * 100
print(f"\n💰 收益提升: {improvement:.1f}%")

# 可视化
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=list(range(1, 10001)),
    y=np.cumsum(ts_revenues),
    mode='lines',
    name='Thompson Sampling',
    line=dict(color=COLORS['blue'], width=2)
))
fig.add_trace(go.Scatter(
    x=list(range(1, 10001)),
    y=np.cumsum(uniform_revenues),
    mode='lines',
    name='均匀推荐',
    line=dict(color=COLORS['gray'], width=2)
))
fig.update_layout(
    title='累积收益对比',
    xaxis_title='展示次数',
    yaxis_title='累积收益 (¥)',
    template='plotly_white',
    height=400
)
fig.show()

---

# 练习与思考题

## 练习 1: Epsilon-Decreasing

实现一个 Epsilon-Decreasing 策略，使得 $\epsilon_t = \min(1, c / t)$，对比与固定 $\epsilon$ 的差异。

**要求**：
- 绘制遗憾曲线对比
- 尝试不同的 $c$ 值
- 分析：为什么递减的 $\epsilon$ 更好？

In [ ]:
# 练习 1: Epsilon-Decreasing 实现

class EpsilonDecreasing:
    def __init__(self, n_arms: int, c: float = 1.0):
        self.n_arms = n_arms
        self.c = c
        self.counts = np.zeros(n_arms)
        self.values = np.zeros(n_arms)
        self.t = 0
    
    def select_arm(self) -> int:
        self.t += 1
        # epsilon_t = min(1, c / t)
        epsilon = min(1.0, self.c / self.t)
        
        if np.random.random() < epsilon:
            return np.random.randint(self.n_arms)
        else:
            return np.argmax(self.values)
    
    def update(self, arm: int, reward: float):
        self.counts[arm] += 1
        n = self.counts[arm]
        self.values[arm] = ((n - 1) / n) * self.values[arm] + (1 / n) * reward
    
    def run(self, true_rates: np.ndarray, n_rounds: int):
        actions = []
        rewards = []
        for _ in range(n_rounds):
            arm = self.select_arm()
            reward = np.random.binomial(1, true_rates[arm])
            self.update(arm, reward)
            actions.append(arm)
            rewards.append(reward)
        return actions, rewards

# 测试不同的 c 值
c_values = [1.0, 5.0, 10.0]
fig = go.Figure()

# 添加固定 epsilon 作为基线
eg_baseline = EpsilonGreedy(n_arms=3, epsilon=0.1)
eg_actions, _ = eg_baseline.run(true_rates, n_rounds)
eg_regret_baseline = []
cumulative = 0
for action in eg_actions:
    cumulative += best_rate - true_rates[action]
    eg_regret_baseline.append(cumulative)

fig.add_trace(go.Scatter(
    x=list(range(1, n_rounds + 1)),
    y=eg_regret_baseline,
    mode='lines',
    name='固定 ε=0.1',
    line=dict(dash='dash', width=2)
))

# 测试递减 epsilon
for c in c_values:
    ed = EpsilonDecreasing(n_arms=3, c=c)
    actions, _ = ed.run(true_rates, n_rounds)
    
    regret = []
    cumulative = 0
    for action in actions:
        cumulative += best_rate - true_rates[action]
        regret.append(cumulative)
    
    fig.add_trace(go.Scatter(
        x=list(range(1, n_rounds + 1)),
        y=regret,
        mode='lines',
        name=f'递减 c={c}',
        line=dict(width=2)
    ))

fig.update_layout(
    title='Epsilon-Decreasing vs 固定 Epsilon',
    xaxis_title='轮数',
    yaxis_title='累积遗憾',
    template='plotly_white',
    height=500
)
fig.show()

print("\n📊 分析：")
print("- 固定 epsilon: 持续探索，遗憾线性增长")
print("- 递减 epsilon: 后期减少探索，遗憾增长变慢")
print("- c 值越大，初期探索越多，后期收敛越慢")

## 练习 2: Batched Thompson Sampling

在真实系统中，模型不是每次都更新，而是积累一批数据后批量更新。实现一个批量版本的 Thompson Sampling。

**要求**：
- 每隔 $B$ 轮更新一次模型（如 $B=100$）
- 对比与实时更新的差异
- 分析：batch size 如何影响性能？

In [ ]:
# 练习 2: Batched Thompson Sampling

class BatchedThompsonSampling:
    def __init__(self, n_arms: int, batch_size: int = 100):
        self.n_arms = n_arms
        self.batch_size = batch_size
        self.alpha = np.ones(n_arms)
        self.beta = np.ones(n_arms)
        
        # 批次缓存
        self.batch_data = {i: [] for i in range(n_arms)}
        self.t = 0
    
    def select_arm(self) -> int:
        samples = np.random.beta(self.alpha, self.beta)
        return np.argmax(samples)
    
    def update(self, arm: int, reward: float):
        self.t += 1
        self.batch_data[arm].append(reward)
        
        # 每 batch_size 轮更新一次
        if self.t % self.batch_size == 0:
            for a in range(self.n_arms):
                if len(self.batch_data[a]) > 0:
                    successes = sum(self.batch_data[a])
                    failures = len(self.batch_data[a]) - successes
                    self.alpha[a] += successes
                    self.beta[a] += failures
                    self.batch_data[a] = []
    
    def run(self, true_rates: np.ndarray, n_rounds: int):
        actions = []
        rewards = []
        for _ in range(n_rounds):
            arm = self.select_arm()
            reward = np.random.binomial(1, true_rates[arm])
            self.update(arm, reward)
            actions.append(arm)
            rewards.append(reward)
        return actions, rewards

# 对比不同 batch size
batch_sizes = [1, 50, 100, 500]
fig = go.Figure()

for bs in batch_sizes:
    if bs == 1:
        # 实时更新 = 标准 Thompson Sampling
        ts_test = ThompsonSampling(n_arms=3)
        actions, _ = ts_test.run(true_rates, n_rounds)
        label = '实时更新'
    else:
        bts = BatchedThompsonSampling(n_arms=3, batch_size=bs)
        actions, _ = bts.run(true_rates, n_rounds)
        label = f'Batch={bs}'
    
    regret = []
    cumulative = 0
    for action in actions:
        cumulative += best_rate - true_rates[action]
        regret.append(cumulative)
    
    fig.add_trace(go.Scatter(
        x=list(range(1, n_rounds + 1)),
        y=regret,
        mode='lines',
        name=label,
        line=dict(width=2)
    ))

fig.update_layout(
    title='Batched vs 实时更新 Thompson Sampling',
    xaxis_title='轮数',
    yaxis_title='累积遗憾',
    template='plotly_white',
    height=500
)
fig.show()

print("\n📊 分析：")
print("- Batch size 越大，更新越不及时，遗憾越大")
print("- 但批量更新可以减少计算开销")
print("- 实际应用中需要权衡性能和计算成本")

## 练习 3: Non-Stationary Bandit

模拟一个非平稳场景：臂的真实收益随时间变化（如季节性波动）。

**要求**：
- 设置某个臂的收益在第 5000 轮时突然提高
- 对比 UCB 和 Thompson Sampling 的适应速度
- 思考：如何改进算法以更快适应变化？

In [ ]:
# 练习 3: Non-Stationary Bandit

def run_nonstationary_experiment(algorithm_class, n_rounds: int = 10000, change_point: int = 5000):
    """运行非平稳实验"""
    # 初始转化率
    rates_phase1 = np.array([0.05, 0.08, 0.06])
    # 第 5000 轮后，臂 0 的转化率提升
    rates_phase2 = np.array([0.09, 0.08, 0.06])  # 臂 0 变成最优
    
    if algorithm_class == 'UCB':
        algo = UCB(n_arms=3)
    else:
        algo = ThompsonSampling(n_arms=3)
    
    actions = []
    rewards = []
    
    for t in range(n_rounds):
        arm = algo.select_arm()
        
        # 根据时间选择转化率
        rates = rates_phase1 if t < change_point else rates_phase2
        reward = np.random.binomial(1, rates[arm])
        
        algo.update(arm, reward)
        actions.append(arm)
        rewards.append(reward)
    
    return actions, rewards

# 运行实验
ucb_actions, _ = run_nonstationary_experiment('UCB')
ts_actions, _ = run_nonstationary_experiment('TS')

# 分析臂 0 的选择比例（滑动窗口）
window = 500

def calculate_arm0_ratio(actions, window_size):
    ratios = []
    for i in range(window_size, len(actions)):
        window_actions = actions[i-window_size:i]
        ratio = sum(1 for a in window_actions if a == 0) / window_size
        ratios.append(ratio)
    return ratios

ucb_arm0 = calculate_arm0_ratio(ucb_actions, window)
ts_arm0 = calculate_arm0_ratio(ts_actions, window)

# 可视化
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=list(range(window, len(ucb_actions))),
    y=ucb_arm0,
    mode='lines',
    name='UCB',
    line=dict(color=COLORS['purple'], width=2)
))

fig.add_trace(go.Scatter(
    x=list(range(window, len(ts_actions))),
    y=ts_arm0,
    mode='lines',
    name='Thompson Sampling',
    line=dict(color=COLORS['blue'], width=2)
))

# 标注变化点
fig.add_vline(x=5000, line_dash="dash", line_color="red",
              annotation_text="臂 0 变为最优")

fig.update_layout(
    title='Non-Stationary Bandit: 算法适应速度对比<br><sub>臂 0 的选择比例（滑动窗口=500）</sub>',
    xaxis_title='轮数',
    yaxis_title='臂 0 选择比例',
    template='plotly_white',
    height=500
)
fig.show()

print("\n📊 分析：")
print("- Thompson Sampling 适应更快（贝叶斯更新灵活）")
print("- UCB 适应较慢（历史数据权重大）")
print("\n改进方法：")
print("- Discounted UCB: 给近期数据更高权重")
print("- Sliding Window: 只使用最近 N 轮数据")
print("- Change Point Detection: 检测环境变化并重置")

## 思考题

### 1. Exploration vs Exploitation

**问题**：在什么情况下应该更多探索？什么情况下应该更多利用？

**提示**：考虑时间、成本、风险等因素。

---

### 2. MAB 的局限

**问题**：为什么 MAB 不能完全替代 A/B Testing？

**提示**：考虑统计推断、因果解释、长期效应等。

---

### 3. Contextual Bandit 的特征工程

**问题**：在推荐系统中，应该如何设计上下文特征？

**提示**：用户特征、物品特征、交叉特征等。

---

### 4. 从 Bandit 到 RL

**问题**：Multi-Armed Bandit 和强化学习（Reinforcement Learning）有什么关系？

**提示**：Bandit 是 RL 的特殊情况（单步决策）。

---

# 总结

## 本节要点

### 核心概念

1. **Explore-Exploit Trade-off**：探索与利用的平衡
2. **Regret**：衡量算法性能的核心指标
3. **Contextual Bandit**：利用上下文信息做个性化决策

### 算法对比

| 算法 | 优点 | 缺点 | 适用场景 |
|-----|------|------|--------|
| **Epsilon-Greedy** | 简单易实现 | 探索盲目 | 快速原型 |
| **UCB** | 理论保证强 | 需要调参 | 需要理论保证 |
| **Thompson Sampling** | 性能最优 | 理论分析难 | 追求性能 |
| **LinUCB** | 利用上下文 | 线性假设 | 个性化推荐 |
| **Neural Bandit** | 建模能力强 | 计算成本高 | 复杂模式 |

### 理论结果

- **UCB**: $R_T = O(\sqrt{KT \log T})$
- **Thompson Sampling**: $R_T = O(\sqrt{KT})$
- **Lower Bound**: $R_T = \Omega\left( \sum_a \frac{\log T}{\Delta_a} \right)$

### 实践应用

- **推荐系统**：新闻、视频、商品推荐
- **广告投放**：动态选择广告
- **动态定价**：电商、打车
- **临床试验**：自适应试验设计

## 进阶阅读

### 经典论文

1. **Auer et al. (2002)**: "Finite-time Analysis of the Multiarmed Bandit Problem" - UCB 算法
2. **Chapelle & Li (2011)**: "An Empirical Evaluation of Thompson Sampling" - Thompson Sampling 实证分析
3. **Li et al. (2010)**: "A Contextual-Bandit Approach to Personalized News Article Recommendation" - LinUCB 在 Yahoo! 的应用

### 书籍

- **Lattimore & Szepesvári (2020)**: "Bandit Algorithms" - 全面的理论教材
- **Sutton & Barto (2018)**: "Reinforcement Learning: An Introduction" - RL 入门，包含 Bandit 章节

### 在线资源

- **Vowpal Wabbit**: 工业级 Contextual Bandit 库
- **Microsoft Decision Service**: 微软的 Bandit 服务

---

## 下一步

- **实践**：在真实项目中应用 MAB（如推荐系统、A/B 测试优化）
- **深入**：学习强化学习（Reinforcement Learning），MAB 是 RL 的基础
- **研究**：探索最新研究（如 Neural Bandit、Offline Bandit Evaluation）

---

恭喜你完成本节学习！🎉

你已经掌握了 Multi-Armed Bandit 的核心思想和主流算法，能够在实际业务中应用这些方法优化决策。

记住：**最好的算法是适合你业务场景的算法**。不要盲目追求理论最优，要考虑实施成本、可解释性、团队能力等因素。

Happy Optimizing! 🚀